In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [17]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [18]:
import tensorflow as tf

In [19]:
import tensorflow.keras.backend as K

In [20]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

In [21]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    print(e)

In [51]:
df = pd.read_csv('edge_images/labels_utf8.csv')

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53480 entries, 0 to 53479
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            53480 non-null  object
 1   Sex           53480 non-null  object
 2   Height        53480 non-null  object
 3   Weight        53480 non-null  object
 4   Hair          53480 non-null  object
 5   Eyes          53480 non-null  object
 6   Race          53480 non-null  object
 7   Sex Offender  53480 non-null  bool  
 8   Offense       53480 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.3+ MB


In [53]:
df[df['Height'].str.split(' ').str[1]!='ft.'].index

Index([40515, 40520, 40525, 40528, 40529, 40530, 40531, 40532, 40536, 40537,
       40538, 40539, 40541, 40542, 40546, 40550, 40552, 40555, 40558, 40563,
       40566, 40569, 40570, 40572, 40573, 40579, 40580, 40582, 40585, 40587,
       40588, 40591, 40592, 40593, 40599, 40604, 40605, 40609, 40610, 40614,
       40619, 40626, 40631, 40635, 40644, 40658, 40661, 40662, 40664, 40665,
       40705, 40728, 40748, 48338, 50288, 53237, 53238, 53239, 53241, 53242,
       53243, 53244],
      dtype='int64')

In [54]:
df.drop(df[df['Height'].str.split(' ').str[1] != 'ft.'].index, axis=0, inplace=True)

In [55]:
df['Height_cm'] = df['Height'].str.split(' ').apply(lambda x: float(x[0])*30.48 + float(x[2])*2.54)

In [56]:
df.drop(df[df['Weight'].str.split(' ').str[1]!= 'lbs.'].index, axis=0, inplace=True)

In [57]:
df['Weight_kg'] = df['Weight'].str.split(' ').apply(lambda x: float(x[0])*0.453592)

In [58]:
df.describe()

,Height_cm,Weight_kg
count,53416.000000,53416.000000
mean,175.885252,85.522047
std,8.470990,17.039862
min,121.920000,7.711064
25%,170.180000,73.481904
50%,175.260000,83.007336
75%,182.880000,95.254320
max,241.300000,250.836376


In [59]:
import os

img_filenames = os.listdir('edge_images/front/front')

img_filenames = pd.Series(img_filenames).apply(lambda x : str(x).split('.')[0])

img_filenames

0        A00147
1        A00198
2        A01072
3        A01077
4        A01148
          ...  
53475    X77379
53476    X77382
53477    X77391
53478    X77410
53479    X77425
Length: 53480, dtype: object

In [60]:
np.any(~df['ID'].isin(img_filenames))

False

In [61]:
df

,ID,Sex,Height,Weight,Hair,Eyes,Race,Sex Offender,Offense,Height_cm,Weight_kg
0,A00147,Male,5 ft. 07 in.,185 lbs.,Brown,Blue,White,True,"AGGR KIDNAPPING-NO RANSOM, ATTEMPT MURDER/INTE...",170.18,83.914520
1,A00198,Male,6 ft. 02 in.,190 lbs.,Red or Auburn,Brown,White,False,"CRIM DMG TO PROP $300-10K, ATTEMPT AGG BATTERY...",187.96,86.182480
2,A01072,Male,5 ft. 09 in.,195 lbs.,Black,Brown,Black,False,"MURDER/INTENT TO KILL/INJURE, CONSP MURDER/INT...",175.26,88.450440
3,A01077,Male,5 ft. 08 in.,180 lbs.,Black,Brown,Black,False,"MURDER/INTENT TO KILL/INJURE, AGG BATTERY/GREA...",172.72,81.646560
4,A01148,Male,5 ft. 11 in.,170 lbs.,Black,Brown,Black,False,"MURDER/INTENT TO KILL/INJURE, ATTEMPT MURDER/I...",180.34,77.110640
...,...,...,...,...,...,...,...,...,...,...,...
53475,Y29993,Male,5 ft. 05 in.,180 lbs.,Black,Brown,Black,False,"RESIDENTIAL BURGLARY, RESIDENTIAL BURGLARY, AT...",165.10,81.646560
53476,Y29994,Male,5 ft. 06 in.,200 lbs.,Black,Brown,Hispanic,False,KIDNAPING/FORCE OR THREAT,167.64,90.718400
53477,Y29996,Male,5 ft. 08 in.,150 lbs.,Black,Brown,Black,False,"RESIDENTIAL BURGLARY, RESIDENTIAL BURGLARY, RE...",172.72,68.038800
53478,Y30000,Male,5 ft. 06 in.,172 lbs.,Brown,Green,White,False,AGG BTRY/CHILD,167.64,78.017824


In [62]:
df.to_csv('edge_images/preprocessed.csv', index=False)